# Basic shape recognition with deep learning

### Using Keras and Tensorflow

Adapted from the [basic_deep_learning_keras](https://github.com/mashgin/basic_deep_learning_keras) tutorial by Amelie Froessl at [Mashgin](http://www.mashgin.com).

Import Python libraries and OpenCV:

In [ ]:
import numpy as np
import random
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(1337)

## Generating sample data
First, we need to generate some sample shape images using OpenCV.

In [ ]:
classes = 2
rows, cols, channels = 100, 100, 1
train_samples = 500
test_samples = int(train_samples * 0.2)

# Generate a single random shape (circle or square)
def generate_shape():
    # Choose a random shape
    cls = random.randint(0, classes - 1)
    img = np.zeros((rows, cols, channels), dtype='uint8')

    # Circle of random size
    if cls == 0:
        cv2.circle(img, (int(cols/2), int(rows/2)),
            random.randint(10, int(cols/2 - 10)), (255), -1)
    # Square of random size
    elif cls == 1:
        side = random.randint(10, cols/2 - 10)
        cv2.rectangle(img, (int(cols/2 - side), int(cols/2 - side)),
                    (int(cols/2 + side), int(cols/2 + side)), (255), -1)

    chans = cv2.split(img)
    out_image = np.array(chans, dtype='float32') / 255.

    return cls, out_image

# Return an array of multiple samples (labels, images)
def generate_samples(batch_size):
    batch_labels, batch_images = [], []

    for i in range(0, batch_size):
        cls, image = generate_shape()
        batch_images.append(image)
        batch_labels.append(cls)

    return np.array(batch_labels).reshape(batch_size, 1), np.array(batch_images)

# Create training and test data
y_train, X_train = generate_samples(train_samples)
y_test, X_test = generate_samples(test_samples)

X_train[0]

The above output is one of the shapes we generated as a 1 channel (black or white) matrix. Below are some samples as images:

In [ ]:
samples = X_train[:5]
for i, x in enumerate(samples):
    img = np.array(x * 255, dtype='uint8')
    plt.subplot(1, 5, i + 1)
    plt.imshow(img[0], cmap='gray')

## Training the model

First, let's import all the Keras methods we'll need.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import backend as K
from keras.models import load_model

Now we can create the model. 

In [ ]:
model = Sequential()

# Define Neural Net layers
model.add(Conv2D(16, (5, 5), input_shape=(rows, cols, channels), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# Convert to probability of each class:
model.add(Dense(classes, activation='softmax'))

Here's just a short description of the layers used in this neural net:

* **Conv2D** → extracting local image information 
* **Activation** → evaluate information relevance
* **MaxPooling2D** → image compression to find most important part of features
* **Dropout** → avoiding bias
* **Flatten** → reformat  
* **Dense** → evaluate global image information (fully connected layer)

And last but not least, this defines how our neural net should learn, which we need for training: 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we can train the model. This should take less than 30 seconds.

In [ ]:
# Prep the data
inputs = X_train.reshape(train_samples, rows, cols, 1)
output_cats = to_categorical(y_train)

model.fit(inputs, output_cats, batch_size=100,
        epochs=4, verbose=1, validation_split=0.2)



The training should have converged to something close to 100% accuracy. But we should still test the model on the test data we created early to make sure it didn't overfit:

In [ ]:
# Prep the data
inputs = X_test.reshape(test_samples, rows, cols, 1)
output_cats = to_categorical(y_test)

accuracy = model.evaluate(inputs, output_cats)[1]

print('Test accuracy = %.2f%%' % (accuracy * 100))

How well did it do? This is a fairly complex model given the simplicity of the data, so it should have done pretty well.

## Making a prediction

Now let's generate a new random shape and see what our model predicts it is.

In [ ]:
label, img = generate_samples(1)
img_test = img.reshape(1, rows, cols, 1)
label_test = to_categorical(label, classes)

img = np.array(img * 255, dtype='uint8')[0]
plt.imshow(img[0], cmap='gray')

# Define our categories
vis_dictionary = {0: "circle", 1: "square"}

pred = model.predict_classes(img_test)
pred = vis_dictionary[pred[0]]
print('Prediction:', pred)

BOOM. You did it. Now you have a neural network model that can recognize the difference between a circle and a square.